# NewEgg.Com WebScraping Program For Laptops - Beta

###  - April 2020

---

In [1]:
# Import dependencies.
import os
import glob
import time
import datetime
import re
from re import search
import pandas as pd
from splinter import Browser
import requests
from bs4 import BeautifulSoup as soup
import random
from playsound import playsound

In [2]:
# Reminder to self.
#import this

## Functions & Classes Setup
---

In [3]:
# Build a function to return date throughout the program.

def return_dt():
    
    global current_date
    
    current_date = str(datetime.datetime.now()).replace(':','.').replace(' ','_')[:-7]
    
    return current_date

#return_dt()

In [4]:
"""
Created a function to ask users if they want to watch the Bot (headless = True/False).
Lastly, will take you directly to the webpage that was inputted.
"""
#head = ''
#browser =''
def head_on_off(executable_path):
    
    # Assigning these as global variables enable us to reference these variables outside of the function
    global head
    global browser
    
    # Bound possible answers.
    
    options = [1, 2]
    
    while head not in options:
        
        head = int(input('Do you want the desktop watch the bot work? Enter a number: 1 - YES | 2 - NO . Your Answer: '))
        
        if head not in options:
            
            print("That was not a valid answer. Please try again. ")
    # For cases where users enter in valid options:
    
    if head == options[0]:
        
        print('Head is activated. Please view only the new automated Google Chrome web browser. ')
        
        print('Do not make any adjustments to this automated window while the program runs, as it may produce errors or undesired outputs. Only time you should touch the automated window when the recaptcha ask you to prove you are a human. ')
        
        browser = Browser('chrome', **executable_path, headless=False, incognito=True)
    
    if head == options[1]:
        
        print('Headless mode activated. No web browser will pop up. Please proceeed. ')
        
        browser = Browser('chrome', **executable_path, headless=True, incognito=True)
    
    # visit the target site
    
    browser.visit(url)

    current_url = browser.url
    
    return current_url

#head_on_off(executable_path)
#time.sleep(5)

In [5]:
"""
Step 5) Previews to the user of the scrappable items-contrainers on the page

"""
# RENAME AS preview***
def preview_y_n(current_page_soup):
    
    global containers
    
    preview = input('Would you like to preview some of the scrappable items off the current page? Enter "y". ')
    
    if preview == "y":
                    
        containers = current_page_soup.find_all("div", class_="item-container")
                    
        print("Preview: expect these scrapped off this page, and for every other total results pages, if there's more than one: ")
        
        print("="*35)
        
        # max items should be 36
        
        counter = 0
        
        for con in containers:
        
            try:
                
                counter += 1
                product_details = con.find_all("a", class_="item-title")[0].text
                
                product_price = con.find_all("li", class_="price-current")[0].text.split()[0]
                
                print(f'{counter}) {product_details} | Price: {product_price}')
                
                print("-"*35)

            except (IndexError) as e:
                
                print(f"{counter}) This item was not scrappable. Skipped. ")
                
                print("-"*35)

        print("="*60)
        
        if counter == 0:
        
            print("Unable to scrap this link. ")
        
        else:
        
            print(f"{len(containers)} Scrappable Objects on the page. ")
    else:
    
        print("Beginning scraping... ")
        
    return None
    
#preview_y_n(current_page_soup)

In [ ]:
"""
Originally modeled out parent/child inheritance object structure.

After careful research, I found it much easier to export the Pandas Dataframe of the results to a dictionary, and then into a class object, which I will elaborate more down below.
"""
# class Product_catalog:
    
#     all_prod_count = 0
    
#     def __init__(self, general_category): # computer systems
#         self.general_category = general_category
        
#         Product_catalog.all_prod_count += 1
        
#     def count_prod(self):
#         return int(self.all_prod_count)
#         #return '{}'.format(self.general_category)

# Sub_category was later changed to Laptops due to the scope of this project.
# class Sub_category(Product_catalog): # laptops/notebooks, gaming
    
#     sub_category_ct = 0
    
#     def __init__(self, general_category, sub_categ, item_num, brand, price, img_link, prod_link, model_specifications, current_promotions):
#         super().__init__(general_category)
#         Sub_category.sub_category_ct += 1
        
#         self.sub_categ = sub_categ
#         self.item_num = item_num
#         self.brand = brand
#         self.price = price
#         self.img_link = img_link
#         self.prod_link = prod_link
#         self.model_specifications = model_specifications
#         self.current_promotions = current_promotions

In [6]:
"""
Main NewEgg WebScraper function.

"""

def newegg_page_scraper(containers, turn_page):
    
    page_nums = []
    general_category = []
    product_categories = []
    images = []
    product_brands = []
    product_models = []
    product_links = []
    item_numbers = []
    promotions = []
    prices = []
    shipping_terms = []
    
    # Put this to avoid error that was being generated
    global gen_category
    
    """ 
    Loop through all the containers on the HTML, and scrap the following content into the following lists
    
    """
    for con in containers:
        
        try:
            page_counter = turn_page
            page_nums.append(int(turn_page))
            
            gen_category = target_page_soup.find_all('div', class_="nav-x-body-top-bar fix")[0].text.split('\n')[5]
            general_category.append(gen_category)
            
            prod_category = target_page_soup.find_all('h1', class_="page-title-text")[0].text
            product_categories.append(prod_category)
            
            image = con.a.img["src"]
            #print(image)
            images.append(image)

            prd_title = con.find_all('a', class_="item-title")[0].text
            product_models.append(prd_title)

            product_link = con.find_all('a', class_="item-title")[0]['href']
            product_links.append(product_link)
            
            shipping = con.find_all('li', class_='price-ship')[0].text.strip().split()[0]
            
            if shipping != "Free":
                shipping = shipping.replace('$', '')
                shipping_terms.append(shipping)
            else:
                shipping = 0.00
                shipping_terms.append(shipping)

            brand_name = con.find_all('a', class_="item-brand")[0].img["title"]
            product_brands.append(brand_name)

        except (IndexError, ValueError) as e:
            
            # If there are no item_brand container, take the Brand from product details.
            product_brands.append(con.find_all('a', class_="item-title")[0].text.split()[0])
            #print(f"{e} block 1")

        try:
            current_promo = con.find_all("p", class_="item-promo")[0].text
            promotions.append(current_promo)
            
        except:
            promotions.append('null')
            #print(f"{e} block 2")
        try:
            price = con.find_all('li', class_="price-current")[0].text.split()[0].replace('$','').replace(',', '')
            prices.append(price)
            
        except:
            price = 'null / out of stock'
            prices.append(price)
            #print(f"{e} block 3")
        
        try:
            item_num = con.find_all('a', class_="item-title")[0]['href'].split('p/')[1].split('?')[0]
            item_numbers.append(item_num)
        except (IndexError) as e:
            item_num = con.find_all('a', class_="item-title")[0]['href'].split('p/')[1]
            item_numbers.append(item_num)    
    
    # Convert all of the lists into a dataframe
    df = pd.DataFrame({
    'item_number': item_numbers,
    'general_category': general_category,
    'product_category': product_categories,
    'brand': product_brands,
    'model_specifications': product_models,
    'price': prices,
    'current_promotions': promotions,
    'shipping': shipping_terms,
    'page_number': page_nums,
    'product_links': product_links,
    'image_link': images
    })
    
    # Rearrange the dataframe columns into the following order.
    df = df[['item_number', 'general_category','product_category', 'page_number' ,'brand','model_specifications' ,'current_promotions' ,'price' ,'shipping' ,'product_links','image_link']]
    
    # Convert the dataframe into a dictionary.
    global scraped_dict
    scraped_dict = df.to_dict('records')
    
    # Grab the subcategory "Laptop/Notebooks" and eliminate any special characters that may cause errors.
    global pdt_category
    pdt_category = df['product_category'].unique()[0]
    # Eliminate special characters in a string if it exists.
    pdt_category = ''.join(e for e in pdt_category if e.isalnum())
    
    """ Count the number of items scraped by getting the length of a all the models for sale.
        This parameter is always available for each item-container in the HTML
    """

    global items_scraped
    items_scraped = len(df['model_specifications'])

    """
    Save the results into a csv file using Pandas
    """
    df.to_csv(f'./processing/{current_date}_{pdt_category}_{items_scraped}_scraped_page{turn_page}.csv')
    
    # Return these variables as they will be used.
    return scraped_dict, items_scraped, pdt_category
    
#df.head()
#newegg_page_scraper(containers, turn_page)

In [7]:
# Function to return the total results pages.

def results_pages(current_page_soup):
    
    # Use BeautifulSoup to extract the total results page number
    results_pages = current_page_soup.find_all('span', class_="list-tool-pagination-text")[0].text.strip()
    #print(results_pages)
    
    # Find and extract total pages + and add 1 to ensure proper length of total pages.
    global total_results_pages
    total_results_pages = int(re.split("/", results_pages)[1]) + 1 # need to add 1 b/c 'range(inclusive, exclusive)'
    #========================================= need to remember to +2, and remove -30
    #print(total_results_pages)
    
    return total_results_pages
#results_pages()

In [8]:
"""
Build a function to concatenate all pages that were scraped and saved in the processing folder.
Save the final output (1 csv file) all the results
    
"""
def concatenate(total_results_pages):
    
    path = f'./processing\\'
    
    scraped_pages = glob.glob(path + "/*.csv")
    
    concatenate_pages = []
    
    counter = 0
    
    for page in scraped_pages:
        
        df = pd.read_csv(page, index_col=0, header=0)
        
        concatenate_pages.append(df)

    compiled_data = pd.concat(concatenate_pages, axis=0, ignore_index=True)
    
    total_items_scraped = len(compiled_data['brand']) # can replace this counter by creating class objects everytime it scrapes
    
    concatenated_output = compiled_data.to_csv(f"./finished_outputs/{current_date}_{total_items_scraped}_scraped_{total_results_pages}_pages_.csv")
    
    return

In [9]:
"""
Built a function to clear out the entire processing files folder to avoid clutter.
Or the user can keep the processing files (page by page) for their own analysis.

"""
def clean_processing_fldr():
    
    path = f'./processing\\'
    
    scraped_pages = glob.glob(path + "/*.csv")
    
    if len(scraped_pages) < 1:
        print("There are no files in the folder to clear. ")
        
    else:
        print(f"Clearing out a total of {len(scraped_pages)} scraped pages in the processing folder... ")
        
        clear_processing_files = []
        
        for page in scraped_pages:
            
            os.remove(page)
        
    print('Clearing of "Processing" folder complete. ')
    
    return

In [ ]:
# def random_a_tag_mouse_over():
    
#     x = random.randint(2, 5)
    
#     def rdm_slp_2_5(x):
        
#         time.sleep(x)

#         print(f"Slept for {x} seconds. ")

#         return x
    
#     # Check to see if there are clickable <a>
    
#     if (browser.is_element_present_by_tag('h1')) == True:
        
#         print("(Check 1) Header is present and hoverable on page. ")
    
#     else:
        
#         print("(Check 1 - ERROR) Header is NOT present on page. ")
        
#         # If there isn't, pause the program. Have user click somewhere on the screen.
#         playsound('./sounds/user_alert.wav')
        
#         red_light = input("Program could not detect a clickable links to hover over, and click. Please use your mouse to click on the background of the screen, and enter 'y' to continue the scrape. ")
        
#     if (browser.is_element_present_by_tag("a")) == True:
        
#         print("(Check 2) <a> tags are present on page. Will begin mouse-over thru the page. ")
        
#     else:
#         # If there isn't, pause the program. Have user click somewhere on the screen.
#         playsound('./sounds/user_alert.wav')
        
#         red_light = input("Program could not detect a clickable links to hover over, and click. Please use your mouse to click on the background of the screen, and enter 'y' to continue the scrape. ")
    

#     # Get the total number of <a> tags off the page.
    
#     number_of_a_tags = len(browser.find_by_tag("a"))
    
#     print(f"Total number of <a>: {number_of_a_tags} ")

#     """
#     My observation found that most of the clickable on screen links to each
#     product was in the 90th to 99th percentile of <a> links.
#     """
#     ninety_pct_a_tag = int(round((number_of_a_tags * .905)))
    
#     ninety_96th_a_tag = int(round((number_of_a_tags * .96)))
    
#     # We know <a> tags are *available* at this point so mouse over through links on the screen.
#     for a_tag_num in range(ninety_pct_a_tag, ninety_96th_a_tag):
        
#         try:
#             # Mouse over odd number <a> on the screen (each page 36 items max). Faster for many result items.
#             if (a_tag_num % 3) != 0:
                
#                 #browser.find_by_tag('h1')[0].click()
                
#                 browser.find_by_tag("a")[a_tag_num].mouse_over()
                
#                 time.sleep(1)
                
#         except:
#             # Mouse over even number <a> on the screen. Faster for less results items.
            
#             print(f"EXEPTION 1 - From random_a_tag_MOUSE_OVER func (Modulo 3 odd) \n Sleeping for 2 seconds and will try again with Modulo 2 - even. ")
            
#             browser.find_by_tag("a")[a_tag_num].mouse_over()
            
#             if (a_tag_num % 2) == 0:
            
#                 browser.find_by_tag("a")[a_tag_num].mouse_over()
#                 time.sleep(1)
                
#     return

In [24]:
def random_a_tag_mouse_over2():
    
    x = random.randint(2, 5)
    
    def rdm_slp_2_5(x):

        time.sleep(x)

        print(f"Slept for {x} seconds. ")

        return
    
    # Check to see if there are clickable <a>
    # Look for for all combinations of text in the laptop section for h1 header
#     if (browser.is_element_present_by_tag('h1')) and \
#     (browser.is_element_present_by_text('Are you a human?') or  
#     )== True:
    
    if (browser.is_element_present_by_tag('h1') and browser.is_element_present_by_text('Laptops')) == True:
        print("(Check 1) Header is present and hoverable on page. ")
    
    else:
        playsound('./sounds/user_alert.wav')
        print("(Check 1 - ERROR) Header is NOT present on page. ")
        
        # If there isn't, pause the program. Have user click somewhere on the screen.
        
        red_light = input("Program could not detect a clickable links to hover over, and click. Please use your mouse to click on the background of the screen, and enter 'y' to continue the scrape. ")
        
    if (browser.is_element_present_by_tag("a")) == True:
        
        print("(Check 2) <a> tags are present on page. Will begin mouse-over thru the page, and click a link. ")
        
    else:
        # If there isn't, pause the program. Have user click somewhere on the screen.
        playsound('./sounds/user_alert.wav')
        red_light = input("Program could not detect a clickable links to hover over, and click. Please use your mouse to click on the background of the screen, and enter 'y' to continue the scrape. ")
    

    # Get the total number of <a> tags off the page.
    
    number_of_a_tags = len(browser.find_by_tag("a"))
    
    print(f"Total number of <a>: {number_of_a_tags} . \n Next will collect working / not working <a> tags")

    """
    My observation found that most of the clickable on screen links to each
    product was in the 90th to 99th percentile of <a> links.
    """
    ninety_pct_a_tag = int(round((number_of_a_tags * .905)))
    
    ninety_96th_a_tag = int(round((number_of_a_tags * .96)))
    
    atag_selected_working = []
    atag_selected_notworking = []
    
    atag_num_list = []
    # list of atags true and false
    atagnum_tf = []
    
    rdm_slp_2_5(x)
    
    # Loop thru all <a> tags between 90.5% - 96%, and find all present <a> tags
    for a_tag_num in range(ninety_pct_a_tag, ninety_96th_a_tag):
    
        # loop thru whole range and make 2 lists: one for atags and one appending results (T/F)
        
        atag_num_list.append(a_tag_num)
        
        result = browser.is_element_present_by_tag("a")[a_tag_num]
        
        atagnum_tf.append(result)
    
    total_atags = len(atagnum_tf)
    
    for a in atagnum_tf:
        
        if a == True:
            # if the <a> tag is present(true), grab the index and store in a list comprehension
            atag_index_true = [i for i, x in enumerate(atagnum_tf) if x]
        
        else:
            # if the <a> tag is NOT present, grab the index and store in a list comprehension
            atag_index_false = [i for i, x in enumerate(atagnum_tf) if x]

    if total_atags == (len(atag_index_true) + len(atag_index_false)):
        
        print(f"{total_atags} <a> tags reconcile. ")
        print(f"Total True: {atag_index_true} | Total False: {atag_index_false} ")
    
    else:
        
        print("Total <a> tags didn't match up to the sum of the true and false - need to investigate. ")
        print(f"{total_atags} Total <a>'s' | Total True: {atag_index_true} | Total False: {atag_index_false} ")
    
    
    atags_mouse_over_success = []
    atags_present_but_errors = []
    
    for a in atag_index_true:
        
        try:
            working_atag_number = atag_num_list[a]
            browser.find_by_tag("a")[working_atag_number].mouse_over()
            time.sleep(1)
        
        except:
            working_atag_number = atag_num_list[a]
            atags_present_but_errors.append(working_atag_number)
            print(f"{working_atag_number} this tag isn't working. Adding to the 'atags_present_but_errors' list. ")
    
    print('Attempting mouse over all present links... ')
        
    return atags_mouse_over_success, atags_present_but_errors
        
    
    ##################################################
    # We know <a> tags are *available* at this point so mouse over through links on the screen.
#     for a_tag_num in range(ninety_pct_a_tag, ninety_96th_a_tag, 3):

#         if (browser.is_element_present_by_tag("a")[a_tag_num]) == True:
        
#             print("(Check 2) <a> tags are present on page. Will begin mouse-over thru the page, and click a link. ")
            
#             print(f"The current a_tag_num is: {a_tag_num} - reference. ")
            
#             atag_selected_working.append(a_tag_num)
            
#         try:
#             # Mouse over odd number <a> on the screen (each page 36 items max). Faster for many result items.
#             #if (a_tag_num % 3) != 0:
                
#                 #browser.find_by_tag('h1')[0].click()
                
#                 browser.find_by_tag("a")[a_tag_num].mouse_over()
                
#                 print("MouseOver&Click Func - Try ran ")
                
#                 time.sleep(1)
                
#         except:
#             # Mouse over even number <a> on the screen. Faster for less results items.
            
#             print(f" EXCEPTION 2 - From random_a_tag_mouse_over_ANDCLICK func (Modulo 3 odd) \n Sleeping for 2 seconds and will try again with Modulo 2 - even. ")
            
#             browser.find_by_tag("a")[a_tag_num].mouse_over()
            
#             print("EXCEPTION 2 MouseOver&Click Func - Mouse over ran, next is click... ")
            
#             if (a_tag_num % 2) == 0:
            
#                 browser.find_by_tag("a")[a_tag_num].mouse_over()
#                 time.sleep(2)
    
#     # After mousing over, thru the page, *click* a random item.
    
#     for a_tag_num in randint(ninety_pct_a_tag, ninety_96th_a_tag):
        
#         if (browser.is_element_present_by_tag("a")[a_tag_num]) == True:
            
#             browser.find_by_tag("a")[a_tag_num].mouse_over()
            
#             browser.find_by_tag("a")[a_tag_num].click()
            
#             return a_tag_num
            
#             rdm_slp_2_5()
            
#             break
            
#         else:
#             # Try again if first time didn't work - could be due to unclickable link.
#             a_tag_num = randint(ninety_pct_a_tag, ninety_96th_a_tag)
            
#             if (browser.is_element_present_by_tag("a")[a_tag_num]) == True:

#                 browser.find_by_tag("a")[a_tag_num].mouse_over()
                
#                 browser.find_by_tag("a")[a_tag_num].click()
                
#                 return a_tag_num
            
#                 rdm_slp_2_5()
                
#                 break
            
#             else:
#                 atag_selected_notworking.append(a_tag_num)
#                 print("Able to mouse over thru page, but was not able to click a link 2 times. Need to review this step. ")
    
#             return atag_selected_working, atag_selected_notworking

In [ ]:
def random_a_tag_mouse_over3():
    
    x = random.randint(2, 5)
    
    def rdm_slp_2_5(x):

        time.sleep(x)

        print(f"Slept for {x} seconds. ")

        return
    
    if (browser.is_element_present_by_tag('h1') and browser.is_element_present_by_text('a')) == True:
        print("(Check 1) Header is present and hoverable on page. ")
    
    else:
        playsound('./sounds/user_alert.wav')
        print("(Check 1 - ERROR) Header is NOT present on page. ")
        
        # If there isn't, pause the program. Have user click somewhere on the screen.

        red_light = input("Program could not detect a clickable links to hover over, and click. Please use your mouse to click on the background of the screen, and enter 'y' to continue the scrape. ")
        

    # Get the total number of <a> tags off the page.
    
    number_of_a_tags = len(browser.find_by_tag("a"))
    
    print(f"Total number of <a>: {number_of_a_tags} . \n Next will collect working / not working <a> tags")

    """
    My observation found that most of the clickable on screen links to each
    product was in the 90th to 99th percentile of <a> links.
    """
    ninety_pct_a_tag = int(round((number_of_a_tags * .905)))
    
    ninety_96th_a_tag = int(round((number_of_a_tags * .96)))
    
    atag_selected_working = []
    atag_selected_notworking = []
    
    atag_num_list = []
    # list of atags true and false
    atagnum_tf = []
    
    rdm_slp_2_5(x)
    
    atags_mouse_over_success = []
    atags_present_but_errors = []
    # Loop thru all <a> tags between 90.5% - 96%, and find all present <a> tags
    for a_tag_num in range(ninety_pct_a_tag, ninety_96th_a_tag, 2):
    
        # loop thru whole range and make 2 lists: one for atags and one appending results (T/F)
        
        atag_num_list.append(a_tag_num)

        try:

            browser.find_by_tag("a")[a_tag_num].mouse_over()
            atags_mouse_over_success.append(a_tag_num)
            time.sleep(1)
        
        except:

            atags_present_but_errors.append(a_tag_num)
            print(f"{a_tag_num} this tag isn't working. Adding to the 'atags_present_but_errors' list. ")
    
    print('Attempting mouse over all present links... ')
        
    return atags_mouse_over_success, atags_present_but_errors

In [65]:

x = random.randint(5, 9)
    
def rdm_slp_6_9(x):

    time.sleep(x)

    #print(f"Slept for {x} seconds. ")

    return x

number_of_a_tags = len(browser.find_by_tag("a"))

ninety_pct_a_tag = int(round((number_of_a_tags * .90))) 

ninety_96th_a_tag = int(round((number_of_a_tags * .95)))

working_try_atags = []

working_except_atags = []

# This will all be working except the last one.
finally_atags = []

working_atags = []

not_working_atags = []

try_counter = 0

except_counter = 0

finally_counter = 0


browser.find_by_tag("h1")

for i in range(ninety_pct_a_tag, ninety_96th_a_tag, 5):
    try:
        browser.find_by_tag("a")[i+2].mouse_over()
        rdm_slp_6_9(x)
        print(f"Mimic Humans - try - Sleep for {x} seconds. ")
        working_try_atags.append(i+2)
        working_atags.append(i+2)
        try_counter += 1
        print(f"{try_counter} - Current Try Count")
        
    except:
        print("Exception - Trying to hover over another link.")
        not_working_atags.append(i+2)
        rdm_slp_6_9(x)
        browser.find_by_tag("a")[i+8].mouse_over()
        working_except_atags.append(i+8)
        working_atags.append(i+8)
        except_counter += 1
        print(f"{except_counter} - Current except Count")
        
    finally:
        not_working_atags.append(i+8)
        print("Finally - Last Attempt")
        # if there's an error, pull the last item in the list from finally_atags list
        finally_atags.append(i+3)
        rdm_slp_6_9(x)
        browser.find_by_tag("a")[i+3].mouse_over()
        working_atags.append(i+3)
        finally_counter += 1
        print(f"{except_counter} - Current finally Count")
        
    print(f"{working_atags[-1]} last working <a> tag")

browser.find_by_tag("h1")
print(f"All working <a> tags: {working_atags} \n")
print(f"All working <a> tags: {not_working_atags} \n")


1 - Current Try Count
Finally - Last Attempt
0 - Current finally Count
1972 last working <a> tag
2 - Current Try Count
Finally - Last Attempt
0 - Current finally Count
1977 last working <a> tag
3 - Current Try Count
Finally - Last Attempt
0 - Current finally Count
1982 last working <a> tag
4 - Current Try Count
Finally - Last Attempt
0 - Current finally Count
1987 last working <a> tag
5 - Current Try Count
Finally - Last Attempt
0 - Current finally Count
1992 last working <a> tag
6 - Current Try Count
Finally - Last Attempt
0 - Current finally Count
1997 last working <a> tag
7 - Current Try Count
Finally - Last Attempt
0 - Current finally Count
2002 last working <a> tag
8 - Current Try Count
Finally - Last Attempt
0 - Current finally Count
2007 last working <a> tag
9 - Current Try Count
Finally - Last Attempt
0 - Current finally Count
2012 last working <a> tag
10 - Current Try Count
Finally - Last Attempt
0 - Current finally Count
2017 last working <a> tag
11 - Current Try Count
Finall

JavascriptException: Message: javascript error: Failed to execute 'elementsFromPoint' on 'Document': The provided double value is non-finite.
  (Session info: chrome=80.0.3987.163)


In [ ]:
# What you need to do:

"""
- Come up with an algorithm that will <a> hop big (assume) 2k plus <a> tag links
- to jump big first, and then except and pause, and then for finally, break the loop after gathering about 5 workable
links, and then set up a another random.randint() function to pick one from the list, mouse_over, and click, and back
to emulate human behavior OR just mouse over and then scrape. and then next page

- Post are you human test <a> links drop down significantly - come up with an algorithm too

- if <a> length is greater than 2k then use 90% method

if <a> is less than 200 use 35% method

"""

In [60]:
print(working_except_atags)

[2005, 2030, 2035]


In [62]:
print(len(working_atags))

33


In [63]:
finally_atags[-1]

2061

In [ ]:
number_of_a_tags

ninety_pct_a_tag

ninety_96th_a_tag

working_try_atags

working_except_atags

finally_atags
working_atags = []

not_working_atags = []

try_counter = 0

except_counter = 0

finally_counter = 0

In [66]:
print(finally_atags[-1])

2032


In [67]:
print(not_working_atags)

[1977, 1982, 1987, 1992, 1997, 2002, 2007, 2012, 2017, 2022, 2027, 2032, 2037]


In [68]:
print(working_atags)

[1971, 1972, 1976, 1977, 1981, 1982, 1986, 1987, 1991, 1992, 1996, 1997, 2001, 2002, 2006, 2007, 2011, 2012, 2016, 2017, 2021, 2022, 2026, 2027, 2031]


In [35]:
browser.is_element_present_by_tag("a")[1989]

TypeError: 'bool' object is not subscriptable

In [46]:
browser.find_link_by_href("href")

[]

In [34]:
browser.is_element_present_by_tag("a")

True

In [44]:
browser.find_by_tag("a")[2100].mouse_over()


In [16]:
t = [True, False, True, False, False, True]

bool_conversion = []
for a in t:
    bool_val = int(a == True)
    bool_conversion.append(bool_val)
    
print(bool_conversion)

[1, 0, 1, 0, 0, 1]


In [22]:
atag_index_true = [i for i, x in enumerate(t) if x]

In [23]:
[i for i, v enumerate(t) if v]
[i for i, x in enumerate(t) if x]

SyntaxError: invalid syntax (<ipython-input-23-db0bf397fa8a>, line 1)

In [ ]:
bool_conversion.index(1)

In [ ]:
t = [False, False, False, False, True, True, False, True, False, False, False, False, False, False, False, False]
[i for i, x in enumerate(t) if x]

In [ ]:
#print(atag_num_list_T_F)

In [ ]:
present_atags = []

for t in bool_conversion:
    #print(t)
     #print(bool_conversion.index(t))
    
    if t == True:
        
        index = atag_num_list_T_F.index(t)
        
        present_atags.append(index,)
            
print(present_atags)

In [ ]:
vowels = ['e', 'e', 'i', 'o', 'i', 'e']

for v in vowels:
    
    a = vowels.index(v)
    print(a)

# index of 'e'
index = vowels.index('e')

print('The index of e:', index)

# index of the first 'i'
index = vowels.index('i')
print('The index of i:', index)

In [25]:
def g_recaptcha_check():
    
    if browser.is_element_present_by_id('g-recaptcha') == True:
        
        playsound('./sounds/user_alert.wav')
        
        continue_scrape = input("Noticed Newegg asked if you were robot. \n When you're done proving you are not a robot and completing all picture tests, enter in any key and press ENTER to continue the webscrape. ")
        
        print("Continuing with scrape... ")
        
        return

    else:
        return

In [26]:
def are_you_human_check():
    
    global current_url
    
    current_url=''
    
    if browser.is_element_present_by_tag('h2') and browser.is_element_present_by_text('Are you a human?')  == True:
        
        playsound('./sounds/user_alert.wav')
        
        continue_scrape = input("Noticed Newegg suspects we are a bot. \n Manually click on a link, act as if you're looking at the item, then go back on the browser. Then enter in any key and press ENTER to continue. ")
        
        # After passes the "Are you human?" test
        if (browser.is_element_present_by_tag('h1') and browser.is_element_present_by_text('Laptops')) == True:
        
            print("(Check 1 - PASSED) Header (Laptops) is present and hoverable on page. - (AFTER ARE YOU HUMAN CHECK) ")

        else:

            print("(Check 1 - ERROR) Header is NOT present on page. - (AFTER ARE YOU HUMAN CHECK) ")
        
            # If there isn't, pause the program. Have user click somewhere on the screen.
            
            playsound('./sounds/user_alert.wav')
            
            print("Program could not detect a clickable links to hover over, and click. ")
            
            continue_scrape = input("Noticed Newegg suspects we are a bot. \n Manually click on a link, act as if you're looking at the item, then go back on the browser. Then enter in any key and press ENTER to continue. ")
        
        current_url = browser.url
        
        return current_url
        
        print('MANUALLY BY-PASSED the "Are you Human?" check by the site. Continuing with scrape... \n')
    
    else:
        
        current_url = browser.url
        
        response = requests.get(current_url)
        
        print(response)
        
        print('Locked in and returning: "current_url" - After the "Are you human?"" check. Proceeding... \n ')
        
        return current_url

In [27]:
def random_xpath_top_bottom():
    
    # Check if there are working links on the screen, otherwise alert the user.
    
    if (browser.is_element_present_by_tag('h1')) == True:
        
        print("(Check 1 - Random Xpath Top Bottom) Header is present and hoverable on page. ")
    
    else:
        
        print("(Check 1 - ERROR - Random Xpath Top Bottom) Header is NOT present on page. ")
        
        # If there isn't, pause the program. Have user click somewhere on the screen.
        
        playsound('./sounds/user_alert.wav')
        
        red_light = input("Program could not detect a clickable links to hover over, and click. Please use your mouse to click on the background of the screen, and enter 'y' to continue the scrape. ")
        
    if (browser.is_element_present_by_tag("a")) == True:
        
        print("(Check 2- Random Xpath Top Bottom) <a> tags are present on page. Will begin mouse-over thru the page, and click a link. ")
        
    else:
        # If there isn't, pause the program. Have user click somewhere on the screen.
        
        playsound('./sounds/user_alert.wav')
        
        red_light = input("Program could not detect a clickable links to hover over, and click. Please use your mouse to click on the background of the screen, and enter 'y' to continue the scrape. ")
    
    # There are clickable links, then 'flip the coin' to choose top or bottom button
    coin_toss_top_bottom = random.randint(0,1)
    
    next_page_button_results = []
    
    # If the coin toss is even, mouse_over and click the top page link.
    if (coin_toss_top_bottom == 0):
        
        print('Heads - Clicking "Next Page" Top Button. ')
        
        browser.find_by_xpath('/html/body/div[4]/section/div/div/div[2]/div/div/div/div[2]/div[1]/div[2]/div[1]/div[2]/div/div[2]/button').mouse_over()
        
        browser.find_by_xpath('/html/body/div[4]/section/div/div/div[2]/div/div/div/div[2]/div[1]/div[2]/div[1]/div[2]/div/div[2]/button').click()
        
        next_page_button_results.append(coin_toss_top_bottom)
        
        print('Heads - SUCCESSFUL "Next Page" Top Button. ')
        
        return
    
    else:
        
        print('Tails - Clicking "Next Page" Bottom Button. ')
        
        browser.find_by_xpath('/html/body/div[4]/section/div/div/div[2]/div/div/div/div[2]/div[1]/div[2]/div[4]/div/div/div[10]/button').mouse_over()
        
        browser.find_by_xpath('/html/body/div[4]/section/div/div/div[2]/div/div/div/div[2]/div[1]/div[2]/div[4]/div/div/div[10]/button').click()
        
        next_page_button_results.append(coin_toss_top_bottom)
        
        print('Tails - SUCCESSFUL "Next Page" Bottom Button. ')
        
        return
    

In [28]:
"""
This class takes in the dictionary from the webscraper function,
and will be used in a list comprehension to produce class "objects"
in the "product_catalog"
"""

class Laptops:
    
    def __init__(self, **entries):
        
        self.__dict__.update(entries)

In [ ]:
# ask if you're in another country
# id="popup_overlay"
# /html/body/div[13]/div/div[2]/div[2]/button[1]

In [ ]:
#browser.is_element_present_by_id()

## Main Program Logic
---

In [29]:
""" Welcome to the program message!
"""

print("=== NewEgg.Com WebScraper Beta ===")
print("=="*30)
print('Scope: This project is a beta and is only built to scrape the laptop section of NewEgg.com due to limited time. ')
print("")
print("Instructions:")
print("")
return_dt()
print(f'Current Date And Time: {current_date}')
print("")
print("(1) Go to www.newegg.com, go to the laptop section, select your requirements (e.g. brand, screensize, and specifications - SSD size, processor brand and etc...) ")
print("(2) Copy and paste the url from your exact search when prompted ")
print('(3) Activate or Disable the "Head View", webscraper bots point of view ')
print('(4) After the webscraping is successful, you will have an option to concatenate all of the pages you scraped together into one csv file')
print('(5) Lastly, you will have an option to clear out the processing folder (data scraped by each page)')
print('(6) If you have any issues or errors, "PRESS CTRL + C" to quit the program in the terminal ')
print('(7) Disclaimer: Newegg may ban you for a 24 - 48 hours for webscraping their data, then you may resume. \n Also, please consider executing during the day, with tons of web traffic to their site in your respective area. ')
print("")


# Set up Splinter requirements.
executable_path = {'executable_path': './chromedriver.exe'}

# Ask user to input in the laptop query link they would like to scrape.
url = input("Step 1) Please copy and paste your laptop query that you want to webscrape, and press enter: ")

# Setting up global variables for head and browser that will used in a function in the program.
head = ''
browser =''

"""
Run the head_on_off function to ask the user if they want to watch the program work.
# head_on_off will finish with browse.visit(url) - and returns "current_url" as an output
that passed into response in a bit below.
"""
head_on_off(executable_path)

"""
Think of safe_proceed_y_n function as a break pedal.
Sometimes it will ask if the user is a bot after multiple scrapes.
"""

safe_proceed_y_n = input(f'The Break Pedal: Answer any robot queries by NewEgg. \n Enter "y" when you are ready to proceed. ')
if safe_proceed_y_n == 'y':
    print(f'Proceeding with webscrape... ')
    current_url = browser.url
    
else:
    print("Quitting browser. \n You will need to press CTRL + C to QUIT the program, and then restart it to try scraping the link again. ")
    browser.quit()

    #AREA WHERE IT BREAKS SOMETIMES - ARE YOU HUMAN
# Gather regquests and gets the data, and parses the text for BeautifulSoup to setup scrape.

"""
Checks to see if Newegg throws a head fake, in the html "Are you human?" to prevent scraping. Returns current_url.
If yes, then we will need to manually bypass, and resume the scrape.
"""

are_you_human_check()

response = requests.get(current_url)
#print(response)

# It breaks here bc of an index error - not the same HTML
current_page_soup = soup(response.text, 'html.parser')

# might need to delete - unnecessary
##current_page_soup.find_all("div", class_="item-container")

# Option to preview some of the data that will be scraped.
preview_y_n(current_page_soup)

# Run the results_pages function to gather the total pages to be scraped.
results_pages(current_page_soup)

time.sleep(2)
"""
This is the loop that performs the page by page scraping of data / results
of the user's query.
"""
# List set up for where class Laptop objects will be stored.
product_catalog = []

for turn_page in range(1, total_results_pages):
    
    """
    If "reCAPTCHA" pops up, pause the program using an input. This allows the user to continue
    to scrape after they're done completing the quiz by inputting any value.
    
    """
    g_recaptcha_check()
        
    try:
        #rdm_slp_4_20()
        print(f"Initial click on screen and product, and then back. ")
        random_a_tag_mouse_over3() # have this return last working index and call it again to continue or just scrape
        # the page
        atags_mouse_over_success = map(int, atags_mouse_over_success)
#         first_atag_num = min(atags_mouse_over_success)
#         last_atag_num = max(atags_mouse_over_success)
        
#         click_atag = random.randint(first_atag_num, last_atag_num)
#         browser.find_by_tag("a")[click_atag].mouse_over()
#         time.sleep(1)
#         browser.find_by_tag("a")[click_atag].click()
#         browser.back()

    # If there's an error, random sleep, try clicking a nonreactive link on the screen, and then click a tag.
    except:
        print(f"EXCEPTION - Couldn't click an <a> tag link...trying another random present <a> link. ")
        random_a_tag_mouse_over3()
        atags_mouse_over_success = map(int, atags_mouse_over_success)
#         first_atag_num = min(atags_mouse_over_success)
#         last_atag_num = max(atags_mouse_over_success)
        
#         click_atag = random.randint(first_atag_num, last_atag_num)
        browser.find_by_tag("a")[click_atag].mouse_over()
        time.sleep(1)
        browser.find_by_tag("a")[click_atag].click()
        browser.back()
        #browser.back()
        
    print("STOPPED PROGRAM. ")
    
    break
    
    # After you press back, run checks again to alert users if we are intercepted.
    g_recaptcha_check()
    # Passing "are_youare_you_human_check" returns "current_url", but will not need it because we have "target_url"
    are_you_human_check()
    
    # target the new page's url for scraping
    target_url = browser.url

    # Use Request.get() - throw the boomerang at the target, retrieve the info, & return back to requestor
    response_target = requests.get(target_url)
    #response

    # Use BeautifulSoup to read grab all the HTML using the lxml parser
    target_page_soup = soup(response_target.text, 'html.parser')
    
    containers = target_page_soup.find_all("div", class_="item-container")

    # Execute webscraper function. Output is a csv file in the processing folder and dictionary.
    newegg_page_scraper(containers, turn_page)

    # Create instances of class objects of the laptops/notebooks using a list comprehension.
    objects = [Laptops(**prod_obj) for prod_obj in scraped_dict]

    # Append all of the objects to the main product_catalog list (List of List of Objects).
    product_catalog.append(objects)

    print("Will scrape pages, but will need to randomly sleep for max 35 seconds to emulate human behavior. ")
    try:
        y = random.randint(1, 5)
        print(f"{turn_page}) | SLEEPING FOR {y} SECONDS")
        time.sleep(y)
        random_a_tag_mouse_over()
        random_xpath_top_bottom()
        
    except:
        y = random.randint(3, 5)
        print(f"{turn_page}) | SLEEPING FOR {y} SECONDS - EXCEPTION")
        time.sleep(y)
        random_a_tag_mouse_over()
        random_xpath_top_bottom()
        time.sleep(1)

# Exit the broswer once complete webscraping
browser.quit()

# Prompt the user if they would like to concatenate all of the pages into one csv file
concat_y_n = input(f'All {total_results_pages} pages have been saved in the "processing" folder (1 page = csv files). Would you like for us concatenate all the files into one? Enter "y", if so. Otherwise, enter anykey to exit the program. ')

if concat_y_n == 'y':
    concatenate(total_results_pages)
    print(f'WebScraping Complete! All {total_results_pages} have been scraped and saved as {current_date}_{pdt_category}_scraped_{total_results_pages}_pages_.csv in the "finished_outputs" folder')

# Prompt the user to if they would like to clear out processing folder function here - as delete everything to prevent clutter
clear_processing_y_n = input(f'The "processing" folder has {total_results_pages} csv files of each page that was scraped. Would you like to clear the files? Enter "y", if so. Otherwise, enter anykey to exit the program. ')
if clear_processing_y_n == 'y':
    clean_processing_fldr()

print('Thank you checking out my project, and hope you found this useful!')

=== NewEgg.Com WebScraper Beta ===
Scope: This project is a beta and is only built to scrape the laptop section of NewEgg.com due to limited time. 

Instructions:

Current Date And Time: 2020-04-20_21.50.15

(1) Go to www.newegg.com, go to the laptop section, select your requirements (e.g. brand, screensize, and specifications - SSD size, processor brand and etc...) 
(2) Copy and paste the url from your exact search when prompted 
(3) Activate or Disable the "Head View", webscraper bots point of view 
(4) After the webscraping is successful, you will have an option to concatenate all of the pages you scraped together into one csv file
(5) Lastly, you will have an option to clear out the processing folder (data scraped by each page)
(6) If you have any issues or errors, "PRESS CTRL + C" to quit the program in the terminal 
(7) Disclaimer: Newegg may ban you for a 24 - 48 hours for webscraping their data, then you may resume. 
 Also, please consider executing during the day, with tons of

TypeError: 'bool' object is not subscriptable

In [ ]:
for 

In [ ]:
current_page_soup


In [ ]:
current_page_soup

In [ ]:
#/html/body/div[5]/section/div/div/div[2]/div/div/div/div[2]/div[1]/div[2]/div[1]/div[2]/div/div[2]/button


In [ ]:
print(browser.find_by_tag('a')[ninety_pct_a_tag].__dict__)

In [ ]:
len(browser.find_by_tag('a')[ninety_pct_a_tag])

In [ ]:
mimipopupsweepstakes

In [ ]:
#browser.find_by_tag('button')#[0].click()

In [ ]:
len(current_page_soup.find_all("div", class_="item-container"))

In [ ]:
#id="rc-imageselect"

In [ ]:
#browser.is_element_present_by_id("btn_InnerSearch")
#browser.is_element_present_by_id("rc-imageselect")

In [ ]:
print(browser.__dict__)

In [ ]:
browser.find_by_tag('body')[0].click()

In [ ]:
len(browser.find_by_tag('body'))

In [ ]:
browser.find_by_tag('a')[1958].click()#['href']

In [ ]:
browser.back()

In [ ]:
number_of_a_tags = len(browser.find_by_tag('a'))
ninety_pct_a_tag = round((number_of_a_tags * .902))
ninety_pct_a_tag

In [ ]:
browser.find_by_tag('a')[1962].click()

In [ ]:
browser.find_by_tag('a')[1970].mouse_over()

In [ ]:
browser.find_by_tag('h1')[0].click()

In [ ]:
#//*[@id="btn_InnerSearch"]

In [ ]:
browser.find_by_xpath('/html/body/div[4]/section/div/div/div[2]/div/div/div/div[2]/div[1]/div[2]/div[1]/div[2]/div/div[2]/button').click()

In [ ]:
browser.is_element_present_by_id('g-recaptcha')

In [ ]:
browser.is_element_present_by_id('mimipopupsweepstakes')


In [ ]:
browser.find_by_tag('a')[2263].click()

In [ ]:
browser.is_element_present_by_id('sweepstakespopup')
/html/body/div[16]/div/a

In [ ]:
browser.is_element_present_by_id('sweepstakespopup_close')

In [ ]:
browser.find_by_id('sweepstakespopup_close').click()

In [ ]:
# browser.find_by_xpath('/html/body/div[4]/section/div/div/div[2]/div/div/div/div[2]/div[1]/div[2]/div[4]/div/div/div[10]/button').click()

In [ ]:
#current_page_soup.is_element_present_by_xpath("/html/body/div[4]/section/div/div/div[2]/div/div/div/div[2]/div[1]/div[2]/div[1]/div[2]/div/div[2]/button")


In [ ]:
#is_element_present_by_xpath('')

In [ ]:
# Test to view objects that were created from the data we collected
#product_catalog[1][0].__dict__

In [ ]:
#/html/body/div[4]/section/div/div/div[2]/div/div/div/div[2]/div[1]/div[2]/div[1]/div[2]/div/div[2]/button/i

In [ ]:
#/html/body/div[4]/section/div/div/div[2]/div/div/div[2]/div[2]/div[1]/div[2]/div[4]/div/div/div[11]/button/i

In [ ]:
counter = 0
prices = []
for con in containers:
    counter += 1
    try:
        #print(str(counter) + " | " + con.find_all('li', class_="price-current")[0].text.split()[0].replace('$','').replace(',', ''))
        price = con.find_all('li', class_="price-current")[0].text.split()[0].replace('$','').replace(',', '')
        prices.append(price)
    except:
        #print(str(counter) + " | " + "null")
        
        price = 'null'
        prices.append(price)
        
print(prices)